<div style="display: flex; background-color: RGB(255,114,0);" >

# PROJET - Bonheur World </h1>
</div>

<div style="display: flex; background-color: Blue; padding: 15px;" >

## 1.Mission 
</div>

Dans cette étude de cas, nous formerons un algorithme d'apprentissage automatique non supervisé pour regrouper les pays en fonction de caractéristiques telles que la production économique, le soutien social, l'espérance de vie, la liberté, l'absence de corruption et la générosité. Le Rapport sur le bonheur dans le monde détermine l'état du bonheur mondial. Les scores de bonheur et les données de classement ont été collectés en demandant aux individus de classer leur vie de 0 (la pire vie possible) à 10 (la meilleure vie possible).         

- Un notebook contenant les fonctions permettant le prétraitement des données ainsi que les résultats du clustering (en y incluant des représentations graphiques) ....
- Un support de présentation qui présente la démarche et les résultats du clustering.

In [62]:
from os import getcwd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from tqdm import tqdm
from bonheur_bed_ara import *

<div style="display: flex; background-color: Blue; padding: 15px;" >

## 2.Chargement des données
</div>

In [63]:
# ---------------------------------------------------------------------------------------------
#                               MAIN
# ---------------------------------------------------------------------------------------------
verbose = False
force_reloading = True

# Récupère le répertoire du programme
file_path = getcwd() + "\\"
data_set_path = file_path + "dataset\\"
country_col_name = "country"
country_official_col_name = "country_official"

print(f"Current execution path : {file_path}")
print(f"Dataset path : {data_set_path}")

Current execution path : c:\Users\User\WORK\workspace-ia\PROJETS\projet_bonheur_bed\
Dataset path : c:\Users\User\WORK\workspace-ia\PROJETS\projet_bonheur_bed\dataset\


<div style="display: flex; background-color: Green; padding: 7px;" >

### 2.1. Données pays
</div>

In [64]:
pays_file_name = "data_set_countries_clean.csv"

df_pays_origin = pd.read_csv(data_set_path+pays_file_name, sep=',')
df_pays_origin = df_pays_origin.sort_values(by=country_col_name)
df_pays_origin["id"] = df_pays_origin["id"].fillna(-1)
df_pays_origin["id"] = df_pays_origin["id"].astype(int)
df_pays_origin = df_pays_origin.reset_index()
df_pays_origin = df_pays_origin.drop("index", axis=1)

print(f"{df_pays_origin.shape} données chargées ------> {list(df_pays_origin.columns)}")
df_pays_origin.head()

(213, 8) données chargées ------> ['id', 'alpha3', 'alpha2', 'country', 'country_official', 'continent_code', 'latitude', 'longitude']


,id,alpha3,alpha2,country,country_official,continent_code,latitude,longitude
0,4,AFG,AF,Afghanistan,Islamic Republic of Afghanistan,AS,29.276770,42.274721
1,8,ALB,AL,Albania,Republic of Albania,EU,21.483333,36.066667
2,12,DZA,DZ,Algeria,People's Democratic Republic of Algeria,AF,14.349729,43.892024
3,24,AGO,AO,Angola,Republic of Angola,AF,41.795972,12.235112
4,10,ATA,AQ,Antarctica,Antarctica,NaN,-22.016981,-47.920533


In [65]:
df_pays_origin.dtypes

id                    int32
alpha3               object
alpha2               object
country              object
country_official     object
continent_code       object
latitude            float64
longitude           float64
dtype: object

In [66]:
df_pays_origin.describe(include="all")

,id,alpha3,alpha2,country,country_official,continent_code,latitude,longitude
count,213.000000,213,212,213,213,182,194.000000,194.000000
unique,NaN,213,212,213,213,5,NaN,NaN
top,NaN,AFG,AF,Afghanistan,Islamic Republic of Afghanistan,AF,NaN,NaN
freq,NaN,1,1,1,1,56,NaN,NaN
mean,338.938967,NaN,NaN,NaN,NaN,NaN,25.128428,19.543371
std,281.280111,NaN,NaN,NaN,NaN,NaN,24.780335,60.798641
min,-1.000000,NaN,NaN,NaN,NaN,NaN,-41.333167,-149.400973
25%,56.000000,NaN,NaN,NaN,NaN,NaN,13.297541,-1.755094
50%,320.000000,NaN,NaN,NaN,NaN,NaN,32.815291,18.894266
75%,578.000000,NaN,NaN,NaN,NaN,NaN,45.811730,45.827753


In [67]:
df_pays_origin[df_pays_origin["alpha2"].duplicated()]

,id,alpha3,alpha2,country,country_official,continent_code,latitude,longitude


In [68]:
# correction des doublons
df_pays_origin.loc[df_pays_origin["alpha3"]=="NAM", "alpha2"] = "NA"
df_pays_origin = df_pays_origin.drop_duplicates(subset=["alpha2"], keep="first")
print(df_pays_origin[df_pays_origin["alpha2"].duplicated()].shape)
df_pays_origin[(df_pays_origin["alpha2"]=="SO") | (df_pays_origin["alpha2"]=="CY")| (df_pays_origin["alpha3"]=="NAM")]

(0, 8)


,id,alpha3,alpha2,country,country_official,continent_code,latitude,longitude
41,196,CYP,CY,Cyprus,Republic of Cyprus,AS,34.683875,32.605168
125,516,NAM,NA,Namibia,Republic of Namibia,AF,34.530234,69.230180
171,706,SOM,SO,Somalia,Federal Republic of Somalia,AF,11.795646,41.012891


In [69]:
df_pays_origin[df_pays_origin["alpha3"].duplicated()]

,id,alpha3,alpha2,country,country_official,continent_code,latitude,longitude


In [70]:
print(df_pays_origin[df_pays_origin["alpha3"].duplicated()].shape)
df_pays_origin[df_pays_origin["alpha3"]=="-99"]

(0, 8)


,id,alpha3,alpha2,country,country_official,continent_code,latitude,longitude


In [71]:
df_pays_origin.loc[df_pays_origin["alpha3"]=="-99", "alpha3"] = np.nan
print(df_pays_origin[df_pays_origin["alpha3"].duplicated()].shape)
df_pays_origin[df_pays_origin["alpha3"].duplicated()]

(0, 8)


,id,alpha3,alpha2,country,country_official,continent_code,latitude,longitude


<div style="display: flex; background-color: Green; padding: 7px;" >

### 2.2. Evolution des scores
</div>

In [72]:
data_evolution_name = "evolution_score_altasocio_2022-04-01.csv"
df_evolution_orgin = pd.read_csv(data_set_path+data_evolution_name, sep=',')
df_evolution_orgin = df_evolution_orgin.rename(columns={"pays":country_col_name})
df_evolution_orgin = df_evolution_orgin.sort_values(by=country_col_name)

# correction des types
df_evolution_orgin = df_correct_type_to_float(df_evolution_orgin, exclude_cols=[country_col_name])

print(f"{df_evolution_orgin.shape} données chargées ------> {list(df_evolution_orgin.columns)}")
df_evolution_orgin.head()

(165, 8) données chargées ------> ['country', 'score_2015', 'score_2016', 'score_2017', 'score_2018', 'score_2019', 'score_2020', 'country_official']


,country,score_2015,score_2016,score_2017,score_2018,score_2019,score_2020,country_official
164,Afghanistan,3.575,3.360,3.794,3.632,3.203,2.567,Islamic Republic of Afghanistan
115,Afrique du Sud,4.642,4.459,4.829,4.724,4.722,4.814,Republic of South Africa
111,Albanie,4.959,4.655,4.644,4.586,4.719,4.883,Republic of Albania
106,Algérie,5.605,6.355,5.872,5.295,5.211,5.005,People's Democratic Republic of Algeria
16,Allemagne,6.750,6.994,6.951,6.965,6.985,7.076,Federal Republic of Germany


In [73]:
df_evolution_orgin.dtypes

country              object
score_2015          float64
score_2016          float64
score_2017          float64
score_2018          float64
score_2019          float64
score_2020          float64
country_official     object
dtype: object

<div style="display: flex; background-color: Green; padding: 7px;" >

### 2.3. Détails des scores
</div>

In [74]:
score_dataset_filenames = []
score_rapport_with ="Rapport-bonheur-"

world_datas_files = []
world_start_with ="world_"

# récupération données
extensions = [".csv", ".xls"]

for ext in extensions:
    score_dataset_filenames.extend(get_dir_files(data_set_path, start_with=score_rapport_with , endwith=ext))
    world_datas_files.extend(get_dir_files(data_set_path, start_with=world_start_with , endwith=ext))

print(f"{len(score_dataset_filenames)} Fichiers score à traiter et {len(world_datas_files)} Fichiers monde à traiter")

4 Fichiers score à traiter et 18 Fichiers monde à traiter


In [61]:
df_origine_light, df_origine_by_line, df_origine = load_scores_files(score_dataset_filenames, data_set_path, country_col_name = country_col_name,country_official_name = country_official_col_name, score_rapport_with=score_rapport_with, verbose=verbose)
df_origine_light.head()

100%|██████████| 4/4 [00:22<00:00,  5.58s/it]


,country_official,Regional indicator,2019-country_origin,2020-country_origin,2021-country_origin,2022-country_origin,2019-country,2019-rank,2022,2021,...,2022-rank,2022-Whisker-high,2022-Whisker-low,2022-Dystopia + residual,2022-PIB,2022-Soutien,2022-Esperance vie BS,2022-Liberte des choix de vie,2022-Generosite,2022-Corruption
0,Arab Republic of Egypt,Middle East and North Africa,Egypt,Egypt,Egypt,Egypt,Egypt,137.0,4.288,4.283,...,129.0,4.377,4.198,0.856,1.388,0.732,0.548,0.469,0.041,0.254
1,Argentine Republic,Latin America and Caribbean,Argentina,Argentina,Argentina,Argentina,Argentina,47.0,5.967,5.929,...,57.0,6.090,5.844,1.891,1.592,1.102,0.662,0.555,0.081,0.085
2,Australia,North America and ANZ,Australia,Australia,Australia,Australia,Australia,11.0,7.162,7.183,...,12.0,7.244,7.081,2.011,1.900,1.203,0.772,0.676,0.258,0.341
3,Bolivia,Latin America and Caribbean,Bolivia,Bolivia,Bolivia,Bolivia,Bolivia,61.0,5.600,5.716,...,71.0,5.709,5.491,2.107,1.256,0.880,0.555,0.627,0.112,0.064
4,Burkina Faso,Sub-Saharan Africa,Burkina Faso,Burkina Faso,Burkina Faso,Burkina Faso,Burkina Faso,115.0,4.670,4.834,...,113.0,4.814,4.527,2.313,0.779,0.565,0.320,0.382,0.186,0.126


In [15]:
# df_temp.loc[df_temp[country_col_name] == "Taiwan Province of China", country_col_name] = "Republic of China"
#         df_temp.loc[df_temp[country_col_name] == "Trinidad & Tobago", country_col_name] = "Trinidad and Tobago"
#         df_temp.loc[df_temp[country_col_name] == "Hong Kong", country_col_name] = "Hong Kong S.A.R. of China"
#         df_temp.loc[df_temp[country_col_name] == "Eswatini, Kingdom of", country_col_name] = "Eswatini"
#         df_temp.loc[df_temp[country_col_name] == "North Cyprus", country_col_name] = "Northern Cyprus"
#         df_temp.loc[df_temp[country_col_name] == "Czechia", country_col_name] = "Czech Republic"

#         # df_temp.loc[df_temp[country_col_name] == "Congo (Kinshasa)", country_col_name] = "Democratic Republic of the Congo"
#         # df_temp.loc[df_temp[country_col_name] == "Congo (Brazzaville)", country_col_name] = "Republic of the Congo"
#         df_temp.loc[df_temp[country_col_name] == "Swaziland", country_col_name] = "Eswatini"

In [23]:
col_y_tp = "2019-country"
col2 = "2020-country"
df_origine_light.loc[(df_origine_light[col_y_tp] != df_origine_light[col2]) &(df_origine_light[col_y_tp].notna()) &(df_origine_light[col2].notna()),['country_official', col_y_tp, col2, 'Regional indicator']]

,country_official,2019-country,2020-country,Regional indicator
24,Hong Kong Special Administrative Region of China,Hong Kong,Hong Kong S.A.R. of China,East Asia
51,Kingdom of Sweden,Swaziland,Sweden,Western Europe
50,Kingdom of Sweden,Swaziland,Sweden,Western Europe
44,Kingdom of Sweden,Sweden,Swaziland,Sub-Saharan Africa
45,Kingdom of Sweden,Sweden,Swaziland,Sub-Saharan Africa
113,Republic of North Macedonia,North Macedonia,Macedonia,Central and Eastern Europe
126,Republic of Trinidad and Tobago,Trinidad & Tobago,Trinidad and Tobago,Latin America and Caribbean


In [27]:
col_y_tp = "2019-country"
df_origine_light.loc[(df_origine_light["country_official"].str.contains("Cyprus")) | (df_origine_light[col_y_tp].str.contains("Cyprus")) ,['country_official', col_y_tp, 'Regional indicator']]

,country_official,2019-country,Regional indicator
162,North Cyprus,NaN,Western Europe
84,Republic of Cyprus,Cyprus,Western Europe
151,The Republic of Cyprus,Northern Cyprus,NaN


In [28]:
col_y_tp = "2020-country"
df_origine_light.loc[(df_origine_light["country_official"].str.contains("Cyprus")) | (df_origine_light[col_y_tp].str.contains("Cyprus")) ,['country_official', col_y_tp, 'Regional indicator']]

,country_official,2020-country,Regional indicator
162,North Cyprus,North Cyprus,Western Europe
84,Republic of Cyprus,Cyprus,Western Europe
151,The Republic of Cyprus,NaN,NaN


In [16]:
col_y_tp = "2020-country_origin"
df_origine_light.loc[(df_origine_light["country_official"].str.contains("Sudan")) | (df_origine_light[col_y_tp].str.contains("Sudan")) ,['country_official', col_y_tp, 'Regional indicator']]

,country_official,2020-country_origin,Regional indicator
118,Republic of South Sudan,South Sudan,Sub-Saharan Africa


In [15]:
df_origine_light.columns

Index(['country_official', 'Regional indicator', '2019-country_origin',
       '2020-country_origin', '2021-country_origin', '2022-country_origin',
       '2019-country', '2019-rank', '2022', '2021', '2020', '2019', '2019-PIB',
       '2019-Soutien', '2019-Esperance vie BS',
       '2019-Liberte des choix de vie', '2019-Generosite', '2019-Corruption',
       '2020-country', '2020-Whisker-high', '2020-Whisker-low', '2020-PIB',
       '2020-Soutien', '2020-Esperance vie BS',
       '2020-Liberte des choix de vie', '2020-Generosite', '2020-Corruption',
       '2020-Dystopia + residual', '2021-country', '2021-Whisker-high',
       '2021-Whisker-low', '2021-PIB', '2021-Soutien', '2021-Esperance vie BS',
       '2021-Liberte des choix de vie', '2021-Generosite', '2021-Corruption',
       '2021-Dystopia + residual', '2022-country', '2022-rank',
       '2022-Whisker-high', '2022-Whisker-low', '2022-Dystopia + residual',
       '2022-PIB', '2022-Soutien', '2022-Esperance vie BS',
       '2022-L

- Une analyse des différences entre les noms de pays des différentes années a été faite pour uniformiser les données. (cf. annexe)
- Puis une fusion et suppression des colonnes inutiles des pays par années (avant de supprimer une vérification a été faite pour identifier les écarts (cf. annexe)

In [16]:
start = 2019
end = 2022

In [17]:
df_origine_light_year_merged = merge_and_clean_country_by_year(df=df_origine_light, start=start, end=end, verbose=verbose)
print(f"BEFORE : {df_origine_light.shape} / AFTER : {df_origine_light_year_merged.shape}")
df_origine_light_year_merged.head()

country_origin na:2 and country na:2
country_origin na:0 and country na:0
country_origin na:0 and country na:0
country_origin na:0 and country na:0


100%|██████████| 4/4 [00:00<?, ?it/s]

BEFORE : (158, 49) / AFTER : (158, 42)


,country_official,country_origin,Regional indicator,2019,2020,2021,2022,2019-rank,2019-Soutien,2022-rank,...,2021-Corruption,2021-Dystopia + residual,2022-Whisker-high,2022-Whisker-low,2022-Dystopia + residual,2022-Soutien,2022-Esperance vie BS,2022-Liberte des choix de vie,2022-Generosite,2022-Corruption
0,Arab Republic of Egypt,Egypt,Middle East and North Africa,4.166,4.151,4.283,4.288,137.0,1.039,129.0,...,0.795,1.648,4.377,4.198,0.856,0.732,0.548,0.469,0.041,0.254
1,Argentine Republic,Argentina,Latin America and Caribbean,6.086,5.975,5.929,5.967,47.0,1.432,57.0,...,0.834,2.461,6.090,5.844,1.891,1.102,0.662,0.555,0.081,0.085
2,Australia,Australia,North America and ANZ,7.228,7.223,7.183,7.162,11.0,1.548,12.0,...,0.442,2.598,7.244,7.081,2.011,1.203,0.772,0.676,0.258,0.341
3,Bolivia,Bolivia,Latin America and Caribbean,5.779,5.747,5.716,5.600,61.0,1.209,71.0,...,0.839,2.805,5.709,5.491,2.107,0.880,0.555,0.627,0.112,0.064
4,Burkina Faso,Burkina Faso,Sub-Saharan Africa,4.587,4.769,4.834,4.670,115.0,1.056,113.0,...,0.748,3.133,4.814,4.527,2.313,0.565,0.320,0.382,0.186,0.126


In [18]:
df_origine_light_year_merged.describe(include="all")

,country_official,country_origin,Regional indicator,2019,2020,2021,2022,2019-rank,2019-Soutien,2022-rank,...,2021-Corruption,2021-Dystopia + residual,2022-Whisker-high,2022-Whisker-low,2022-Dystopia + residual,2022-Soutien,2022-Esperance vie BS,2022-Liberte des choix de vie,2022-Generosite,2022-Corruption
count,158,158,153,156.000000,153.000000,149.000000,146.000000,156.000000,156.000000,146.000000,...,149.000000,149.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000
unique,158,158,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Arab Republic of Egypt,Egypt,Sub-Saharan Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,5.407096,5.473255,5.532839,5.553575,78.500000,1.208814,73.500000,...,0.727450,2.430329,5.673589,5.433568,1.831808,0.905863,0.586171,0.517226,0.147377,0.154781
std,NaN,NaN,NaN,1.113120,1.112288,1.073924,1.086843,45.177428,0.299191,42.290661,...,0.179226,0.537645,1.065621,1.109380,0.534994,0.280122,0.176336,0.145859,0.082799,0.127514
min,NaN,NaN,NaN,2.853000,2.567000,2.523000,2.404000,1.000000,0.000000,1.000000,...,0.082000,0.648000,2.469000,2.339000,0.187000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,4.544500,4.724000,4.852000,4.888750,39.750000,1.055750,37.250000,...,0.667000,2.138000,5.006250,4.754750,1.555250,0.732000,0.463250,0.440500,0.089000,0.068250
50%,NaN,NaN,NaN,5.379500,5.515000,5.534000,5.568500,78.500000,1.271500,73.500000,...,0.781000,2.509000,5.680000,5.453000,1.894500,0.957500,0.621500,0.543500,0.132500,0.119500
75%,NaN,NaN,NaN,6.184500,6.228000,6.255000,6.305000,117.250000,1.452500,109.750000,...,0.845000,2.794000,6.448750,6.190000,2.153000,1.114250,0.719750,0.626000,0.197750,0.198500


<div style="display: flex; background-color: indigo;" >

#### 2.3.x. Complément des données
</div>

In [19]:
df_origine_light_completed = df_origine_light_year_merged.copy()

<div style="display: flex; background-color: Green; padding: 7px;" >

### 2.4. Fusion Détails des scores et évolution des scores
</div>

In [20]:
df_light_merged = df_origine_light_completed.merge(df_evolution_orgin, on=country_official_col_name, how="outer", indicator=True)
df_light_merged = df_light_merged.rename(columns={"country":"country_FR"})
print(df_origine_light_completed.shape, df_evolution_orgin.shape, df_light_merged.shape)
print(list(df_light_merged.columns))
df_light_merged = df_light_merged.sort_values(by=country_official_col_name)
df_light_merged = df_light_merged.reset_index()
df_light_merged = df_light_merged.drop("index", axis=1)
df_light_merged.head()

(158, 42) (165, 8) (167, 50)
['country_official', 'country_origin', 'Regional indicator', '2019', '2020', '2021', '2022', '2019-rank', '2019-Soutien', '2022-rank', '2019-PIB', '2020-PIB', '2022-PIB', '2021-PIB', '2019-Esperance vie BS', '2019-Liberte des choix de vie', '2019-Generosite', '2019-Corruption', '2020-Whisker-high', '2020-Whisker-low', '2020-Soutien', '2020-Esperance vie BS', '2020-Liberte des choix de vie', '2020-Generosite', '2020-Corruption', '2020-Dystopia + residual', '2021-Whisker-high', '2021-Whisker-low', '2021-Soutien', '2021-Esperance vie BS', '2021-Liberte des choix de vie', '2021-Generosite', '2021-Corruption', '2021-Dystopia + residual', '2022-Whisker-high', '2022-Whisker-low', '2022-Dystopia + residual', '2022-Soutien', '2022-Esperance vie BS', '2022-Liberte des choix de vie', '2022-Generosite', '2022-Corruption', 'country_FR', 'score_2015', 'score_2016', 'score_2017', 'score_2018', 'score_2019', 'score_2020', '_merge']


,country_official,country_origin,Regional indicator,2019,2020,2021,2022,2019-rank,2019-Soutien,2022-rank,...,2022-Generosite,2022-Corruption,country_FR,score_2015,score_2016,score_2017,score_2018,score_2019,score_2020,_merge
0,Arab Republic of Egypt,Egypt,Middle East and North Africa,4.166,4.151,4.283,4.288,137.0,1.039,129.0,...,0.041,0.254,Égypte,4.194,4.362,4.735,4.419,4.166,4.151,both
1,Argentine Republic,Argentina,Latin America and Caribbean,6.086,5.975,5.929,5.967,47.0,1.432,57.0,...,0.081,0.085,Argentine,6.574,6.650,6.599,6.388,6.086,5.975,both
2,Australia,Australia,North America and ANZ,7.228,7.223,7.183,7.162,11.0,1.548,12.0,...,0.258,0.341,Australie,7.284,7.313,7.284,7.272,7.228,7.223,both
3,Belize,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Belize,NaN,5.956,5.956,5.956,NaN,NaN,right_only
4,Bolivia,Bolivia,Latin America and Caribbean,5.779,5.747,5.716,5.600,61.0,1.209,71.0,...,0.112,0.064,Bolivie,5.890,5.822,5.823,5.752,5.779,5.747,both


In [21]:
# Réorganisation des colonnes
cols_names = list(df_light_merged.columns)
cols_names.remove('country_FR')
init_pos = 3
for i in range(2015, 2021, 1):
    c_n = 'score_'+str(i)
    if i > 2018:
        c_n2 = str(i)
        cols_names.remove(c_n2)
        cols_names.insert(init_pos, c_n2)
        init_pos += 1
    cols_names.remove(c_n)
    cols_names.insert(init_pos, c_n)
    init_pos += 1
    
# score_2015	score_2016	score_2017	score_2018	score_2019	score_2020
cols_names.insert(3, 'country_FR')
df_light_merged = df_light_merged[cols_names]
df_light_merged[['country_official', 'country_origin', 'Regional indicator',
       'country_FR', 'score_2015', 'score_2016', 'score_2017', 'score_2018',
       '2019', 'score_2019', '2020', 'score_2020', '2021', '2022','_merge']].head()

,country_official,country_origin,Regional indicator,country_FR,score_2015,score_2016,score_2017,score_2018,2019,score_2019,2020,score_2020,2021,2022,_merge
0,Arab Republic of Egypt,Egypt,Middle East and North Africa,Égypte,4.194,4.362,4.735,4.419,4.166,4.166,4.151,4.151,4.283,4.288,both
1,Argentine Republic,Argentina,Latin America and Caribbean,Argentine,6.574,6.650,6.599,6.388,6.086,6.086,5.975,5.975,5.929,5.967,both
2,Australia,Australia,North America and ANZ,Australie,7.284,7.313,7.284,7.272,7.228,7.228,7.223,7.223,7.183,7.162,both
3,Belize,NaN,NaN,Belize,NaN,5.956,5.956,5.956,NaN,NaN,NaN,NaN,NaN,NaN,right_only
4,Bolivia,Bolivia,Latin America and Caribbean,Bolivie,5.890,5.822,5.823,5.752,5.779,5.779,5.747,5.747,5.716,5.600,both


In [22]:
df_light_merged.loc[(df_light_merged["country_origin"].isna()) & (df_light_merged["score_2019"].notna()) ,['country_official', 'country_origin', 'Regional indicator',
       'country_FR', 'score_2015', 'score_2016', 'score_2017', 'score_2018',
       '2019', 'score_2019', '2020', 'score_2020', '2021', '2022','_merge']]

,country_official,country_origin,Regional indicator,country_FR,score_2015,score_2016,score_2017,score_2018,2019,score_2019,2020,score_2020,2021,2022,_merge
124,Republic of South Sudan,NaN,NaN,Soudan du Sud,NaN,3.832,3.591,3.254,NaN,2.853,NaN,2.817,NaN,NaN,right_only


In [23]:
df_light_merged.loc[(df_light_merged["country_official"].str.contains("Sudan")) | (df_light_merged["country_origin"].str.contains("Sudan")) ,['country_official', 'country_origin', 'Regional indicator',
       'country_FR', 'score_2015', 'score_2016', 'score_2017', 'score_2018',
       '2019', 'score_2019', '2020', 'score_2020', '2021', '2022','_merge']]

,country_official,country_origin,Regional indicator,country_FR,score_2015,score_2016,score_2017,score_2018,2019,score_2019,2020,score_2020,2021,2022,_merge
124,Republic of South Sudan,NaN,NaN,Soudan du Sud,NaN,3.832,3.591,3.254,NaN,2.853,NaN,2.817,NaN,NaN,right_only
139,Republic of the Sudan,South Sudan,Sub-Saharan Africa,Soudan,4.55,4.139,4.139,4.139,2.853,NaN,2.817,NaN,NaN,NaN,both


In [48]:
col1 = '2019'
col2 = 'score_'+col1

diff = df_light_merged[(df_light_merged[col1] != df_light_merged[col2]) & (df_light_merged[col2].notna()) & (df_light_merged[col1].notna())]
print(diff.shape)
diff[['country_official', 'country_origin', 'Regional indicator',
       'country_FR', col1, col2,'_merge']]

(3, 50)


,country_official,country_origin,Regional indicator,country_FR,2019,score_2019,_merge
131,Republic of the Congo,Congo (Brazzaville),Sub-Saharan Africa,Congo (RDC),4.812,4.418,both
140,Slovak Republic,Slovakia,Central and Eastern Europe,Slovénie,6.198,6.118,both
143,State of Kuwait,Kuwait,Middle East and North Africa,Koweït,6.021,6.011,both


In [51]:
df_light_merged[df_light_merged["country_official"].str.contains("Congo")]

,country_official,country_origin,Regional indicator,country_FR,score_2015,score_2016,score_2017,score_2018,2019,score_2019,...,2021-Dystopia + residual,2022-Whisker-high,2022-Whisker-low,2022-Dystopia + residual,2022-Soutien,2022-Esperance vie BS,2022-Liberte des choix de vie,2022-Generosite,2022-Corruption,_merge
8,Democratic Republic Of The Congo,Congo (Kinshasa),Sub-Saharan Africa,NaN,NaN,NaN,NaN,NaN,4.418,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
130,Republic of the Congo,Congo (Brazzaville),Sub-Saharan Africa,Congo,3.989,4.272,4.291,4.559,4.812,4.812,...,3.476,5.235,4.915,2.658,0.405,0.355,0.431,0.13,0.146,both
131,Republic of the Congo,Congo (Brazzaville),Sub-Saharan Africa,Congo (RDC),4.517,4.236,4.280,4.245,4.812,4.418,...,3.476,5.235,4.915,2.658,0.405,0.355,0.431,0.13,0.146,both


In [47]:
col1 = '2020'
col2 = 'score_'+col1

diff = df_light_merged[(df_light_merged[col1] != df_light_merged[col2]) & (df_light_merged[col2].notna()) & (df_light_merged[col1].notna())]
print(diff.shape)
diff[['country_official', 'country_origin', 'Regional indicator',
       'country_FR', col1, col2,'_merge']]

(2, 50)


,country_official,country_origin,Regional indicator,country_FR,2020,score_2020,_merge
131,Republic of the Congo,Congo (Brazzaville),Sub-Saharan Africa,Congo (RDC),5.194,4.311,both
140,Slovak Republic,Slovakia,Central and Eastern Europe,Slovénie,6.281,6.363,both


In [40]:
df_light_merged.columns

Index(['country_official', 'country_origin', 'Regional indicator',
       'country_FR', 'score_2015', 'score_2016', 'score_2017', 'score_2018',
       '2019', 'score_2019', '2020', 'score_2020', '2021', '2022', '2019-rank',
       '2019-Soutien', '2022-rank', '2019-PIB', '2020-PIB', '2022-PIB',
       '2021-PIB', '2019-Esperance vie BS', '2019-Liberte des choix de vie',
       '2019-Generosite', '2019-Corruption', '2020-Whisker-high',
       '2020-Whisker-low', '2020-Soutien', '2020-Esperance vie BS',
       '2020-Liberte des choix de vie', '2020-Generosite', '2020-Corruption',
       '2020-Dystopia + residual', '2021-Whisker-high', '2021-Whisker-low',
       '2021-Soutien', '2021-Esperance vie BS',
       '2021-Liberte des choix de vie', '2021-Generosite', '2021-Corruption',
       '2021-Dystopia + residual', '2022-Whisker-high', '2022-Whisker-low',
       '2022-Dystopia + residual', '2022-Soutien', '2022-Esperance vie BS',
       '2022-Liberte des choix de vie', '2022-Generosite', '2

In [ ]:
for i in range(2019, 2021, 1):
    df_origine_light_merged['score_'+str(i)] = df_origine_light_merged['score_'+str(i)].fillna(df_origine_light_merged[str(i)])

In [ ]:
df_origine_light_merged.columns

In [ ]:
df_origine_light_merged.loc[df_origine_light_merged["_merge"]!="both", ['country', 'country_FR', 'country_official', 'Regional indicator',
       'score_2015', 'score_2016', 'score_2017', 'score_2018', 'score_2019','2019',
       'score_2020',  '2020', '2021', '2022']]

Il y a 5 pays qui n'ont pas de score entre 2015 et 2020

In [ ]:
print(df_origine_light_merged.loc[df_origine_light_merged["score_2019"] != df_origine_light_merged["2019"],['country', 'country_FR', 'country_official', 'Regional indicator','score_2019','2019']].shape)
df_origine_light_merged.loc[df_origine_light_merged["score_2019"] != df_origine_light_merged["2019"],['country', 'country_FR', 'country_official', 'Regional indicator','score_2019','2019']]

In [ ]:
print(df_origine_light_merged.loc[df_origine_light_merged["score_2020"] != df_origine_light_merged["2020"],['country', 'country_FR', 'country_official', 'Regional indicator','score_2020', '2020']].shape)
df_origine_light_merged.loc[df_origine_light_merged["score_2020"] != df_origine_light_merged["2020"],['country', 'country_FR', 'country_official', 'Regional indicator','score_2020', '2020']]

In [ ]:
df_origine_light_merged.loc[df_origine_light_merged["score_2019"] != df_origine_light_merged["2019"],['country', 'country_FR', 'country_official', 'Regional indicator',
       'score_2019','2019','score_2020', '2020']].shape

<div style="display: flex; background-color: Green; padding: 7px;" >

### 2.1. Visualisation des données
</div>

<div style="display: flex; background-color: indigo;" >

#### 2.1.1 DF données fournies > 2019
</div>

In [ ]:
df_origine.describe(include='all')

In [ ]:
df_origine = df_origine.rename(columns={'Country or region':'country', 
                                        'Score':2019,
                                        'GDP per capita':'2019-PIB', 
                                        'Social support':'2019-Soutien',
                                        'Healthy life expectancy':'2019-Esperance vie BS',
                                        'Freedom to make life choices':'2019-Liberte des choix de vie',
                                        'Generosity':'2019-Generosite',
                                        'Perceptions of corruption':'2019-Corruption'
                                        })
df_origine.columns

<div style="display: flex; background-color: indigo;" >

#### 2.1.2 DF temporelle
</div>

In [ ]:
df_evolution_orgin.head()

In [ ]:
df_evolution_orgin = df_evolution_orgin.rename(columns={"pays":"country"})
df_evolution_orgin.columns

In [ ]:
df_evolution_orgin.describe(include="all")

<div style="display: flex; background-color: indigo;" >

#### 2.1.3 DF données pays
</div>

In [ ]:
df_pays_origin = df_pays_origin.rename(columns={"id":"id_country", 'country_official':'official_name'})
df_pays_origin.columns

In [ ]:
print(df_pays_origin.shape)
df_pays_origin.describe(include="all")

<div style="display: flex; background-color: Green; padding: 7px;" >

### 2.2. Fusion des DF
</div>

<div style="display: flex; background-color: indigo;" >

#### 2.2.1. Fusion dataset source et données temporelles
</div>

In [ ]:
print(f"ORIGIN : {df_origine.shape}, {df_evolution_orgin.shape}, AFTER ", end="")
df_global = df_origine.merge(df_evolution_orgin, on='country', how="outer", indicator=True)
print(df_global.shape)
df_global.head()

In [ ]:
df_global[df_global["_merge"]!="both"]

<div style="display: flex; background-color: indigo;" >

#### 2.2.2. Fusion dataset intermédiaire et données pays
</div>

In [ ]:
print(f"ORIGIN : {df_origine.shape}, {df_pays_origin.shape}, AFTER ", end="")
df_global = df_origine.merge(df_pays_origin, on='country', how="left", indicator=True)
print(df_global.shape)
df_global.head()

In [ ]:
df_global[df_global["_merge"]!="both"]

<div style="display: flex; background-color: Blue; padding: 15px;" >

## 3.Exploration des données
</div>

<div style="display: flex; background-color: green;" >

## 1.1. Visualisation des données
</div>


In [ ]:
df_origine.shape

In [ ]:
df_origine.head()

In [ ]:
df_origine.dtypes

In [ ]:
df_origine.describe()

In [ ]:
df_origine.hist(figsize=(25,25),bins=50)

<div style="display: flex; background-color: green;" >

## 1.2. Analyse des NaN
</div>


In [ ]:
sns.heatmap(df_origine.isnull(), yticklabels=False,cbar=False, cmap='viridis')

In [ ]:
msno.heatmap(df_origine)

In [ ]:
# Avant nettoyage
msno.matrix(df_origine)

In [ ]:
msno.bar(df_origine)

<div style="display: flex; background-color: green;" >

## 1.2. Corrélations entre les données
</div>


In [ ]:
df_origine["score_int"] = round(df_origine["Score"])
df_origine["score_int"] = df_origine["score_int"].astype('int')

In [ ]:
sns.pairplot(df_origine, hue="score_int")

In [ ]:
draw_correlation_graphe(df_origine, "Corrélation entre les variables", verbose=verbose,fontsize=12)

<div style="display: flex; background-color: green;" >

## 1.3. Visualisation des outliers
</div>


In [ ]:
cols = get_numeric_columns_names(df_origine)
for column_name in cols:
    q_low, q_hi,iqr, q_min, q_max = get_outliers_datas(df_origine, column_name)
    graphe_outliers(df_origine, column_name, q_min, q_max)

<div style="display: flex; background-color: green;" >

## 1.4. Analyse des autres doublons possible
</div>


# Chargement DF pays

In [ ]:
pays_file_name = "data_set_countries_clean.csv"

<div style="display: flex; background-color: Blue; padding: 15px;" >

## Annexes
</div>

In [ ]:
just to fail

<div style="display: flex; background-color: Green; padding: 7px;" >

### Titre 2
</div>

<div style="display: flex; background-color: indigo;" >

#### Titre 3
</div>

<div style="display: flex; background-color: Green; padding: 7px;" >

### Nettoyage des pays par année dans la DF 
</div>

In [ ]:
diff = df_origine_light[df_origine_light["2019-country_origin"] != df_origine_light["2019-country"]]
diff = diff[['country_official', 'Regional indicator', '2019-country_origin','2019-country',
       '2020-country_origin', '2020-country', '2021-country_origin', '2021-country', '2022-country_origin', '2022-country' ]]
print(diff[diff["2019-country"].notna()].shape)
diff[diff["2019-country"].notna()]

(6, 10)


,country_official,Regional indicator,2019-country_origin,2019-country,2020-country_origin,2020-country,2021-country_origin,2021-country,2022-country_origin,2022-country
8,Democratic Republic Of The Congo,Sub-Saharan Africa,Congo (Kinshasa),Democratic Republic of the Congo,Congo (Kinshasa),Democratic Republic of the Congo,NaN,NaN,NaN,NaN
24,Hong Kong Special Administrative Region of China,East Asia,Hong Kong,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China
40,Kingdom of Eswatini,Sub-Saharan Africa,Swaziland,Eswatini,Swaziland,Eswatini,Swaziland,Eswatini,Eswatini. Kingdom of,Eswatini
119,Republic of Trinidad and Tobago,Latin America and Caribbean,Trinidad & Tobago,Trinidad and Tobago,Trinidad and Tobago,Trinidad and Tobago,NaN,NaN,NaN,NaN
127,Republic of the Congo,Sub-Saharan Africa,Congo (Brazzaville),Republic of the Congo,Congo (Brazzaville),Republic of the Congo,Congo (Brazzaville),Republic of the Congo,Congo,Congo
131,Republic of the Sudan,Sub-Saharan Africa,South Sudan,Sudan,South Sudan,Sudan,NaN,NaN,NaN,NaN


In [ ]:
diff = df_origine_light[df_origine_light["2019-country_origin"] != df_origine_light["2020-country_origin"]]
diff = diff[['country_official', 'Regional indicator', '2019-country_origin',
       '2020-country_origin', '2019-country','2020-country', '2021-country_origin', '2021-country', '2022-country_origin', '2022-country' ]]
print(diff[diff["2019-country_origin"].notna() & diff["2020-country_origin"].notna()].shape)
diff[diff["2019-country_origin"].notna() & diff["2020-country_origin"].notna()]

(4, 10)


,country_official,Regional indicator,2019-country_origin,2020-country_origin,2019-country,2020-country,2021-country_origin,2021-country,2022-country_origin,2022-country
24,Hong Kong Special Administrative Region of China,East Asia,Hong Kong,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China
107,Republic of North Macedonia,Central and Eastern Europe,North Macedonia,Macedonia,North Macedonia,Macedonia,North Macedonia,North Macedonia,North Macedonia,North Macedonia
119,Republic of Trinidad and Tobago,Latin America and Caribbean,Trinidad & Tobago,Trinidad and Tobago,Trinidad and Tobago,Trinidad and Tobago,NaN,NaN,NaN,NaN
145,The Republic of Cyprus,Western Europe,Northern Cyprus,North Cyprus,Northern Cyprus,Northern Cyprus,North Cyprus,Northern Cyprus,North Cyprus,Northern Cyprus


In [ ]:
diff = df_origine_light[df_origine_light["2020-country_origin"] != df_origine_light["2021-country_origin"]]
diff = diff[['country_official', 'Regional indicator', 
       '2020-country_origin', '2021-country_origin', '2020-country', '2021-country', '2022-country_origin', '2022-country' ]]
print(diff[diff["2020-country_origin"].notna() & diff["2021-country_origin"].notna()].shape)
diff[diff["2020-country_origin"].notna() & diff["2021-country_origin"].notna()]

(1, 8)


,country_official,Regional indicator,2020-country_origin,2021-country_origin,2020-country,2021-country,2022-country_origin,2022-country
107,Republic of North Macedonia,Central and Eastern Europe,Macedonia,North Macedonia,Macedonia,North Macedonia,North Macedonia,North Macedonia


In [ ]:
diff = df_origine_light[df_origine_light["2021-country_origin"] != df_origine_light["2022-country_origin"]]
diff = diff[['country_official', 'Regional indicator', 
       '2021-country_origin', '2022-country_origin', '2021-country', '2022-country' ]]
print(diff[diff["2021-country_origin"].notna() & diff["2022-country_origin"].notna()].shape)
diff[diff["2021-country_origin"].notna() & diff["2022-country_origin"].notna()]

(3, 6)


,country_official,Regional indicator,2021-country_origin,2022-country_origin,2021-country,2022-country
7,Czech Republic,Central and Eastern Europe,Czech Republic,Czechia,Czech Republic,Czech Republic
40,Kingdom of Eswatini,Sub-Saharan Africa,Swaziland,Eswatini. Kingdom of,Eswatini,Eswatini
127,Republic of the Congo,Sub-Saharan Africa,Congo (Brazzaville),Congo,Republic of the Congo,Congo


In [ ]:
diff = df_origine_light[df_origine_light["2020-country_origin"] != df_origine_light["2020-country"]]
diff = diff[['country_official', 'Regional indicator', '2019-country_origin','2019-country',
       '2020-country_origin', '2020-country', '2021-country_origin', '2021-country', '2022-country_origin', '2022-country' ]]
print(diff[diff["2020-country"].notna()].shape)
diff[diff["2020-country"].notna()]

(6, 10)


,country_official,Regional indicator,2019-country_origin,2019-country,2020-country_origin,2020-country,2021-country_origin,2021-country,2022-country_origin,2022-country
8,Democratic Republic Of The Congo,Sub-Saharan Africa,Congo (Kinshasa),Democratic Republic of the Congo,Congo (Kinshasa),Democratic Republic of the Congo,NaN,NaN,NaN,NaN
40,Kingdom of Eswatini,Sub-Saharan Africa,Swaziland,Eswatini,Swaziland,Eswatini,Swaziland,Eswatini,Eswatini. Kingdom of,Eswatini
156,Republic of China,East Asia,NaN,NaN,Taiwan Province of China,Republic of China,Taiwan Province of China,Republic of China,Taiwan Province of China,Republic of China
127,Republic of the Congo,Sub-Saharan Africa,Congo (Brazzaville),Republic of the Congo,Congo (Brazzaville),Republic of the Congo,Congo (Brazzaville),Republic of the Congo,Congo,Congo
131,Republic of the Sudan,Sub-Saharan Africa,South Sudan,Sudan,South Sudan,Sudan,NaN,NaN,NaN,NaN
145,The Republic of Cyprus,Western Europe,Northern Cyprus,Northern Cyprus,North Cyprus,Northern Cyprus,North Cyprus,Northern Cyprus,North Cyprus,Northern Cyprus


In [ ]:
diff = df_origine_light_completed2[df_origine_light_completed2["2020-country_origin"] != df_origine_light_completed2["country_origin"]]
diff = diff[['country_official', 'Regional indicator', 'country_origin','country',
       '2020-country_origin' ]]
diff

,country_official,Regional indicator,country_origin,country,2020-country_origin
16,Federal Republic of Somalia,NaN,Somalia,Somalia,NaN
24,Hong Kong Special Administrative Region of China,East Asia,Hong Kong,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China
37,Kingdom of Bhutan,NaN,Bhutan,Bhutan,NaN
107,Republic of North Macedonia,Central and Eastern Europe,North Macedonia,North Macedonia,Macedonia
119,Republic of Trinidad and Tobago,Latin America and Caribbean,Trinidad & Tobago,Trinidad and Tobago,Trinidad and Tobago
140,State of Qatar,NaN,Qatar,Qatar,NaN
142,Syria,NaN,Syria,Syria,NaN
143,Taiwan,NaN,Taiwan,Taiwan,NaN
145,The Republic of Cyprus,Western Europe,Northern Cyprus,Northern Cyprus,North Cyprus


In [ ]:
try:
    df_origine_light_completed2 = df_origine_light_completed2.drop("2020-country_origin", axis=1)
except:
    pass
diff = df_origine_light_completed2[df_origine_light_completed2["2021-country_origin"] != df_origine_light_completed2["country_origin"]]
diff = diff[['country_official', 'Regional indicator', 'country_origin','country',
       '2021-country_origin' ]]
diff

,country_official,Regional indicator,country_origin,country,2021-country_origin
6,Central African Republic,Sub-Saharan Africa,Central African Republic,Central African Republic,NaN
8,Democratic Republic Of The Congo,Sub-Saharan Africa,Congo (Kinshasa),Democratic Republic of the Congo,NaN
16,Federal Republic of Somalia,NaN,Somalia,Somalia,NaN
24,Hong Kong Special Administrative Region of China,East Asia,Hong Kong,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China
37,Kingdom of Bhutan,NaN,Bhutan,Bhutan,NaN
119,Republic of Trinidad and Tobago,Latin America and Caribbean,Trinidad & Tobago,Trinidad and Tobago,NaN
131,Republic of the Sudan,Sub-Saharan Africa,South Sudan,Sudan,NaN
140,State of Qatar,NaN,Qatar,Qatar,NaN
142,Syria,NaN,Syria,Syria,NaN
143,Taiwan,NaN,Taiwan,Taiwan,NaN


In [ ]:
try:
    df_origine_light_completed2 = df_origine_light_completed2.drop("2021-country_origin", axis=1)
except:
    pass
diff = df_origine_light_completed2[df_origine_light_completed2["2022-country_origin"] != df_origine_light_completed2["country_origin"]]
diff = diff[['country_official', 'Regional indicator', 'country_origin','country',
       '2022-country_origin' ]]
diff

,country_official,Regional indicator,country_origin,country,2022-country_origin
6,Central African Republic,Sub-Saharan Africa,Central African Republic,Central African Republic,NaN
7,Czech Republic,Central and Eastern Europe,Czech Republic,Czech Republic,Czechia
8,Democratic Republic Of The Congo,Sub-Saharan Africa,Congo (Kinshasa),Democratic Republic of the Congo,NaN
16,Federal Republic of Somalia,NaN,Somalia,Somalia,NaN
24,Hong Kong Special Administrative Region of China,East Asia,Hong Kong,Hong Kong S.A.R. of China,Hong Kong S.A.R. of China
37,Kingdom of Bhutan,NaN,Bhutan,Bhutan,NaN
40,Kingdom of Eswatini,Sub-Saharan Africa,Swaziland,Eswatini,Eswatini. Kingdom of
72,Republic of Burundi,Sub-Saharan Africa,Burundi,Burundi,NaN
87,Republic of Haiti,Latin America and Caribbean,Haiti,Haiti,NaN
157,Republic of Maldives,South Asia,Maldives,Maldives,NaN


In [ ]:
col = "2020-country"
diff = df_origine_light_completed2[df_origine_light_completed2[col] != df_origine_light_completed2["country"]]
diff = diff[['country_official', 'Regional indicator', 'country_origin','country', col ]]
diff

,country_official,Regional indicator,country_origin,country,2020-country
16,Federal Republic of Somalia,NaN,Somalia,Somalia,NaN
37,Kingdom of Bhutan,NaN,Bhutan,Bhutan,NaN
107,Republic of North Macedonia,Central and Eastern Europe,North Macedonia,North Macedonia,Macedonia
140,State of Qatar,NaN,Qatar,Qatar,NaN
142,Syria,NaN,Syria,Syria,NaN
143,Taiwan,NaN,Taiwan,Taiwan,NaN


In [ ]:
col = "2021-country"
diff = df_origine_light_completed2[df_origine_light_completed2[col] != df_origine_light_completed2["country"]]
diff = diff[['country_official', 'Regional indicator', 'country_origin','country', col ]]
diff

,country_official,Regional indicator,country_origin,country,2021-country
6,Central African Republic,Sub-Saharan Africa,Central African Republic,Central African Republic,NaN
8,Democratic Republic Of The Congo,Sub-Saharan Africa,Congo (Kinshasa),Democratic Republic of the Congo,NaN
16,Federal Republic of Somalia,NaN,Somalia,Somalia,NaN
37,Kingdom of Bhutan,NaN,Bhutan,Bhutan,NaN
119,Republic of Trinidad and Tobago,Latin America and Caribbean,Trinidad & Tobago,Trinidad and Tobago,NaN
131,Republic of the Sudan,Sub-Saharan Africa,South Sudan,Sudan,NaN
140,State of Qatar,NaN,Qatar,Qatar,NaN
142,Syria,NaN,Syria,Syria,NaN
143,Taiwan,NaN,Taiwan,Taiwan,NaN


In [ ]:
col = "2022-country"
diff = df_origine_light_completed2[df_origine_light_completed2[col] != df_origine_light_completed2["country"]]
diff = diff[['country_official', 'Regional indicator', 'country_origin','country', col ]]
diff

,country_official,Regional indicator,country_origin,country,2022-country
6,Central African Republic,Sub-Saharan Africa,Central African Republic,Central African Republic,NaN
8,Democratic Republic Of The Congo,Sub-Saharan Africa,Congo (Kinshasa),Democratic Republic of the Congo,NaN
16,Federal Republic of Somalia,NaN,Somalia,Somalia,NaN
37,Kingdom of Bhutan,NaN,Bhutan,Bhutan,NaN
72,Republic of Burundi,Sub-Saharan Africa,Burundi,Burundi,NaN
87,Republic of Haiti,Latin America and Caribbean,Haiti,Haiti,NaN
157,Republic of Maldives,South Asia,Maldives,Maldives,NaN
119,Republic of Trinidad and Tobago,Latin America and Caribbean,Trinidad & Tobago,Trinidad and Tobago,NaN
127,Republic of the Congo,Sub-Saharan Africa,Congo (Brazzaville),Republic of the Congo,Congo
131,Republic of the Sudan,Sub-Saharan Africa,South Sudan,Sudan,NaN


In [ ]:
df_origine_light_completed2[df_origine_light_completed2["country_origin"]!=df_origine_light_completed2["country"]]

,country_official,Regional indicator,country_origin,country,2019-rank,2022,2021,2020,2019,2019-PIB,...,2022-rank,2022-Whisker-high,2022-Whisker-low,2022-Dystopia + residual,2022-PIB,2022-Soutien,2022-Esperance vie BS,2022-Liberte des choix de vie,2022-Generosite,2022-Corruption
8,Democratic Republic Of The Congo,Sub-Saharan Africa,Congo (Kinshasa),Democratic Republic of the Congo,127.0,NaN,NaN,4.311,4.418,0.094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,Hong Kong Special Administrative Region of China,East Asia,Hong Kong,Hong Kong S.A.R. of China,76.0,5.425,5.477,5.510,5.430,1.438,...,81.0,5.501,5.348,0.642,1.957,0.954,0.942,0.400,0.147,0.383
40,Kingdom of Eswatini,Sub-Saharan Africa,Swaziland,Eswatini,135.0,4.396,4.308,4.308,4.212,0.811,...,125.0,4.607,4.185,1.688,1.274,0.786,0.197,0.259,0.038,0.154
156,Republic of China,East Asia,Taiwan Province of China,Republic of China,NaN,6.512,6.584,6.455,NaN,NaN,...,26.0,6.596,6.429,2.002,1.897,1.095,0.733,0.542,0.075,0.168
119,Republic of Trinidad and Tobago,Latin America and Caribbean,Trinidad & Tobago,Trinidad and Tobago,39.0,NaN,NaN,6.192,6.192,1.231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,Republic of the Congo,Sub-Saharan Africa,Congo (Brazzaville),Republic of the Congo,103.0,5.075,5.342,5.194,4.812,0.673,...,99.0,5.235,4.915,2.658,0.950,0.405,0.355,0.431,0.130,0.146
131,Republic of the Sudan,Sub-Saharan Africa,South Sudan,Sudan,156.0,NaN,NaN,2.817,2.853,0.306,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div style="display: flex; background-color: Green; padding: 7px;" >

### Nettoyage des données suite fusion avec les fichiers d'évolution des scores par année
</div>

In [ ]:
col1 = '2019'
col2 = 'score_'+col1

diff = df_light_merged[(df_light_merged[col1] != df_light_merged[col2]) & (df_light_merged[col2].notna()) & (df_light_merged[col1].notna())]
print(diff.shape)
diff[['country_official', 'country_origin', 'Regional indicator',
       'country_FR', col1, col2,'_merge']]

(3, 50)


,country_official,country_origin,Regional indicator,country_FR,2019,score_2019,_merge
131,Republic of the Congo,Congo (Brazzaville),Sub-Saharan Africa,Congo (RDC),4.812,4.418,both
140,Slovak Republic,Slovakia,Central and Eastern Europe,Slovénie,6.198,6.118,both
143,State of Kuwait,Kuwait,Middle East and North Africa,Koweït,6.021,6.011,both


In [ ]:
col1 = '2020'
col2 = 'score_'+col1

diff = df_light_merged[(df_light_merged[col1] != df_light_merged[col2]) & (df_light_merged[col2].notna()) & (df_light_merged[col1].notna())]
print(diff.shape)
diff[['country_official', 'country_origin', 'Regional indicator',
       'country_FR', col1, col2,'_merge']]

(2, 50)


,country_official,country_origin,Regional indicator,country_FR,2020,score_2020,_merge
131,Republic of the Congo,Congo (Brazzaville),Sub-Saharan Africa,Congo (RDC),5.194,4.311,both
140,Slovak Republic,Slovakia,Central and Eastern Europe,Slovénie,6.281,6.363,both
